In [1]:
# general
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import importlib

%matplotlib inline

from pylab import rcParams
rcParams['figure.figsize'] = 8, 5

# first used in exercise one
import linearsvm as svm
from sklearn import preprocessing # for scale
import scipy.linalg
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

In [2]:
#importlib.reload(svm)

Note: Per the request in the "Collaboration policy" note, I've discussed at least part of this assignment with many of the MS employees in the class, including Abhishek, Geoff, Suman, and Charles. (Different weeks/different assignments have different people, depending upon who attends our study groups, but I'll probably just include this blurb w/ each homework since it's generally correct.) I've also gotten input from the discussion board.

# Exercise one

_Compute the gradient ∇F(β) of F._

![Gradient](gradient.png)

_Consider the Spam dataset from The Elements of Statistical Learning. Standardize the data, if you have not done so already._

In [3]:
spam = pd.read_table('data/spam.data', sep=' ', header=None)
spam.shape

(4601, 58)

In [4]:
spam[:2]

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.0,0.00,...,0.0,0.000,0.0,0.778,0.00,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.0,0.94,...,0.0,0.132,0.0,0.372,0.18,0.048,5.114,101,1028,1


In [5]:
# and the train/test split info
spam_traintest = pd.read_table('data/spam.traintest', 
                               header=None, names=['TestIndicator'])
spam_traintest.shape

(4601, 1)

In [6]:
spam_traintest[:3]

,TestIndicator
0,1
1,0
2,1


In [7]:
# convert label 0/1 to -1/1
spam[57] = spam[57].apply(lambda v: -1 if v == 0 else 1)

In [8]:
spam[57].value_counts(dropna=False)

-1    2788
 1    1813
Name: 57, dtype: int64

In [9]:
X = spam.values[:,0:57]
y = spam.values[:,57]
X.shape, y.shape

((4601, 57), (4601,))

In [10]:
X_scaled = preprocessing.scale(X)
X_scaled.shape

(4601, 57)

In [11]:
X_scaled_train = X_scaled[spam_traintest['TestIndicator'] == 0, :]
X_scaled_test = X_scaled[spam_traintest['TestIndicator'] == 1, :]
y_train = y[spam_traintest['TestIndicator'] == 0]
y_test = y[spam_traintest['TestIndicator'] == 1]

X_scaled_train.shape, X_scaled_test.shape, y_train.shape, y_test.shape

((3065, 57), (1536, 57), (3065,), (1536,))

_Write a function mylinearsvm that implements the fast gradient algorithm to train the linear support vector machine with the squared hinge loss. The function takes as input the initial step-size value for the backtracking rule and a maximum number of iterations._

I implemented the mylinearsvm function, and all of the supporting functions including the gradient and objective functions, in the file linearsvm.py, which I imported into this notebook with the alias svm. Instead of creating a function named mylinearsvm, I just call my fastgradalgo function and pass in references to my linear SVM gradient and objective functions. I also include my (relatively minimal) unit tests, in linearsvm-test.py.

_Train your linear support vector machine with the squared hinge loss on the the Spam dataset for the λ = 1. Report your misclassiﬁcation error for this value of λ._ 

As shown below, my misclassification error is 9.6% (corresponding to an accuracy of 90.4%).

In [12]:
t_init = 0.01
max_iters = 1000

In [13]:
results = svm.fastgradalgo(
    X_scaled_train, y_train, t_init=t_init, 
    grad_func = svm.compute_linearsvm_gradient, 
    obj_func = svm.compute_linearsvm_objective, 
    lam=1, max_iter=max_iters)
results[-3:]

,0,1,2,3,4,5,6,7,8,9,...,47,48,49,50,51,52,53,54,55,56
998,0.004707,-0.017785,0.041067,0.022033,0.063291,0.054787,0.113581,0.060567,0.043423,0.024483,...,-0.023849,-0.023435,-0.017923,-0.011564,0.057493,0.105909,0.022018,0.024618,0.054175,0.056466
999,0.004707,-0.017785,0.041067,0.022033,0.063291,0.054787,0.113581,0.060567,0.043423,0.024483,...,-0.023849,-0.023435,-0.017923,-0.011564,0.057493,0.105909,0.022018,0.024618,0.054175,0.056466
1000,0.004707,-0.017785,0.041067,0.022033,0.063291,0.054787,0.113581,0.060567,0.043423,0.024483,...,-0.023849,-0.023435,-0.017923,-0.011564,0.057493,0.105909,0.022018,0.024618,0.054175,0.056466


In [14]:
svm.get_final_coefs(results)

array([[ 0.00470651, -0.01778492,  0.04106708,  0.02203334,  0.06329117,
         0.05478699,  0.11358142,  0.06056665,  0.04342264,  0.02448297,
         0.04303622, -0.01988106,  0.01359624,  0.01196979,  0.03772923,
         0.10491026,  0.06641436,  0.0520275 ,  0.05424548,  0.04369458,
         0.09507983,  0.04484736,  0.09426391,  0.06432963, -0.05355806,
        -0.03838683, -0.04609687, -0.01992538, -0.01869907, -0.02547454,
        -0.00801355, -0.00424814, -0.02992379, -0.00442401, -0.01561272,
        -0.00625731, -0.03108503, -0.01270518, -0.02830545,  0.00894927,
        -0.01613486, -0.03341811, -0.0241412 , -0.02498623, -0.0456691 ,
        -0.04229804, -0.01272238, -0.02384934, -0.02343515, -0.01792313,
        -0.01156379,  0.05749315,  0.10590906,  0.02201775,  0.02461781,
         0.05417492,  0.05646572]])

In [15]:
def get_accuracy(beta_coefs, X, y_actual, threshold=0):
    """
    Return the classification accuracy given a set of coefficients, in 
    beta_coefs, and observations, in X, compared to actual/known values 
    in y_actual. The threshold parameter defines the value above which the
    predicted value is considered a positive example.
    """
    y_pred = X.dot(beta_coefs.T).ravel() # ravel to convert to vector
    # convert to -1 or +1 depending on threshold
    y_thresholded = np.where(y_pred > threshold, 1, -1)
    return accuracy_score(y_actual, y_thresholded)

In [16]:
# note use of the held out test data to get the performance metrics
accuracy = get_accuracy(svm.get_final_coefs(results), X_scaled_test, y_test)
print("Accuracy: {0:.1%}".format(accuracy))
print("Misclassification error: {0:.1%}".format(1 - accuracy))

Accuracy: 90.4%
Misclassification error: 9.6%


_Run cross-validation to ﬁnd the optimal value of λ. Report your misclassiﬁcation error for that value of λ._

As shown below, my optimal value of $\lambda$ is zero. The misclassification error from the test/hold out set using coefficients from a model trained with a lambda of zero is 8.4% (which corresponds to an accuracy of 91.6%).

In [17]:
def train_and_test_single_fold(X_full, y_full, lam, train_index, test_index):
    """
    Train using the data identified by the indices in train_index, and then test
    (and return accuracy) using the data identified by the indices in test_index.
    """
    beta_vals = svm.fastgradalgo(
        X_full[train_index], y_full[train_index], t_init=t_init, 
        grad_func = svm.compute_linearsvm_gradient, 
        obj_func = svm.compute_linearsvm_objective, 
        lam=lam, max_iter=max_iters)
    
    final_coefs = svm.get_final_coefs(beta_vals)
    
    return get_accuracy(final_coefs, X_full[test_index], y_full[test_index])

In [18]:
def train_and_test_for_all_folds(X_full, y_full, train_indices, 
                                 test_indices, lam):
    """
    Train and test for all folds - for now, 10 folds, hard-coded. Return 
    the mean of the set of accuracy scores from all folds.
    """
    accuracy_scores = [train_and_test_single_fold(X_full, y_full, lam,
                                       train_indices[i], 
                                       test_indices[i]) for i in range(10)]
    return(np.mean(accuracy_scores))

In [19]:
# get arrays with 10 sets of test and train indices - one for each fold
kf = KFold(10, shuffle=True, random_state=42)

train_indices_list = []
test_indices_list = []
for train_index, test_index in kf.split(X_scaled_train):
    train_indices_list.append(train_index)
    test_indices_list.append(test_index)
    
train_indices = np.array(train_indices_list)
test_indices = np.array(test_indices_list)

In [20]:
lambdas = [10 ** exponent for exponent in range(-5,2)]
lambdas

[1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10]

In [21]:
# and finally, do 10-fold cross validation for each value of lambda, and
# show the mean of each set's classification accuracy
accuracy_values_by_lambda = [train_and_test_for_all_folds(X_scaled_train, 
                                y_train, train_indices, test_indices, 
                                lam) for lam in lambdas]
list(zip(lambdas, accuracy_values_by_lambda))

[(1e-05, 0.9161567775861702),
 (0.0001, 0.91583104468714738),
 (0.001, 0.91583210917374558),
 (0.01, 0.91191692746588315),
 (0.1, 0.91027974707798442),
 (1, 0.90929083902833663),
 (10, 0.8939569095825084)]

In [22]:
best_lambda = 0

In [24]:
# train with best lambda
results_best_lambda = svm.fastgradalgo(
    X_scaled_train, y_train, t_init=t_init, 
    grad_func = svm.compute_linearsvm_gradient, 
    obj_func = svm.compute_linearsvm_objective, 
    lam=best_lambda, max_iter=max_iters)
results_best_lambda[-3:]

,0,1,2,3,4,5,6,7,8,9,...,47,48,49,50,51,52,53,54,55,56
998,-0.005867,-0.027225,0.050431,4.278140,0.102371,0.071049,0.308368,0.091116,0.047523,0.021685,...,-0.189956,-0.179221,-0.062239,-0.016350,0.073153,0.617278,0.258428,-0.144212,0.473202,0.094983
999,-0.005865,-0.027226,0.050431,4.284186,0.102372,0.071052,0.308329,0.091115,0.047523,0.021686,...,-0.190031,-0.179161,-0.062229,-0.016322,0.073152,0.617243,0.258434,-0.144224,0.473228,0.094986
1000,-0.005863,-0.027228,0.050432,4.290233,0.102373,0.071055,0.308291,0.091114,0.047523,0.021687,...,-0.190106,-0.179100,-0.062220,-0.016295,0.073151,0.617208,0.258439,-0.144237,0.473254,0.094989


In [25]:
accuracy = get_accuracy(svm.get_final_coefs(results_best_lambda), 
                        X_scaled_test, y_test)
print("Accuracy: {0:.1%}".format(accuracy))
print("Misclassification error: {0:.1%}".format(1 - accuracy))

Accuracy: 91.6%
Misclassification error: 8.4%
